In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import os
import cv2
from sklearn.preprocessing import label_binarize
from itertools import cycle

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-10-28 14:45:33.040040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761662733.276209      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761662733.342915      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# --- Parameters ---
DATASET_PATH = "/kaggle/input/augmented-alzheimer-dataset/AugmentedAlzheimerDataset"
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
NUM_CLASSES = 4 # Mild, VeryMild, Non, Moderate


# Create the training dataset (80% of the data)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale' # Using grayscale as the image appears to be single-channel
)

# Create the validation dataset (20% of the data)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)


class_names = train_dataset.class_names
print("Found classes:", class_names)


# Normalize pixel values from [0, 255] to [0, 1]
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))


AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

NotFoundError: Could not find directory /kaggle/input/augmented-alzheimer-dataset/AugmentedAlzheimerDataset

In [ ]:
# Custom Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, units):
        super(SelfAttention, self).__init__()
        self.W1 = layers.Dense(units, activation='tanh')
        self.W2 = layers.Dense(1)

    def call(self, inputs):
        
        score = self.W2(self.W1(inputs)) # (batch_size, sequence_length, 1)
        
       
        attention_weights = tf.nn.softmax(score, axis=1)
        
        
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1) # (batch_size, hidden_units)
        
        return context_vector

# Model Definition with CNN + BiLSTM
def build_cnn_bilstm_attention_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    last_conv = layers.Conv2D(128, (3, 3), padding='same', activation='relu', name='last_conv')(x)
    
    
    reshaped = layers.Reshape((32, 32 * 128))(last_conv)
    
    
    bilstm = layers.Bidirectional(
        layers.LSTM(units=128, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))
    )(reshaped)
    
  
    attention_output = SelfAttention(units=128)(bilstm)
    
    
    dropout1 = layers.Dropout(0.5)(attention_output)
    
    
    dense1 = layers.Dense(128, activation='relu')(dropout1)
    dropout2 = layers.Dropout(0.5)(dense1)
    outputs = layers.Dense(num_classes, activation='softmax')(dropout2)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


input_shape = (IMG_HEIGHT, IMG_WIDTH, 1)
model = build_cnn_bilstm_attention_model(input_shape, NUM_CLASSES)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:

EPOCHS = 30


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


y_pred_probs = model.predict(validation_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.concatenate([y for x, y in validation_dataset], axis=0)


print("\\nClassification Report:\\n")
print(classification_report(y_true, y_pred, target_names=class_names))


conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# --- ROC AUC Curve Visualization ---

y_true_binarized = label_binarize(y_true, classes=range(NUM_CLASSES))


fpr = dict()
tpr = dict()
roc_auc = dict()


for i in range(NUM_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


plt.figure(figsize=(10, 8))


colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green'])

for i, color in zip(range(NUM_CLASSES), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))


plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')


plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Multi-Class')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:

model.save("dementia_cnn_bilstm_attention_model.h5")
print("Model saved successfully as dementia_cnn_bilstm_attention_model.h5")

In [ ]:
# --- Grad-CAM Implementation ---
def compute_gradcam(model, img_array, class_idx, layer_name='last_conv'):
    
    grad_model = Model(inputs=model.inputs, outputs=[model.get_layer(layer_name).output, model.output])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_idx]
    
    
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    
    conv_outputs = conv_outputs[0]
    
    
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0)
    heatmap = heatmap / tf.reduce_max(heatmap + 1e-10) 
    return heatmap.numpy()

def overlay_heatmap(heatmap, img_array, alpha=0.6, colormap=cv2.COLORMAP_INFERNO):
   
    if len(heatmap.shape) > 2:
        heatmap = heatmap.squeeze()

    
    heatmap = cv2.resize(heatmap, (img_array.shape[2], img_array.shape[1]), interpolation=cv2.INTER_LINEAR)

    
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap + 1e-10)

   
    heatmap[heatmap < 0.5] = 0

    
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)

   
    img = np.uint8(255 * img_array[0, ..., 0])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    
    superimposed_img = cv2.addWeighted(heatmap, alpha, img_rgb, 1 - alpha, 0)
    return superimposed_img


def load_and_preprocess_image(image_path, target_size=(128, 128)):
    
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not load image from {image_path}")
    
  
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
    
   
    img = img.astype(np.float32) / 255.0
    
   
    img = np.expand_dims(img, axis=(0, -1))
    return img

#image path put here ............
image_path = "/kaggle/input/augmented-alzheimer-dataset/AugmentedAlzheimerDataset/severeDementia/003c36ee-8c43-41ce-b5e5-90622644bba5.jpg"  


sample_img = load_and_preprocess_image(image_path)
print("Sample image shape:", sample_img.shape)


preds = model.predict(sample_img)
pred_class_idx = np.argmax(preds[0])
pred_class_name = class_names[pred_class_idx]


heatmap = compute_gradcam(model, sample_img, pred_class_idx)


superimposed_img = overlay_heatmap(heatmap, sample_img)
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(sample_img[0, ..., 0], cmap='gray')
plt.subplot(1, 2, 2)
plt.title(f'Grad-CAM: {pred_class_name}')
plt.imshow(superimposed_img)
plt.show()